## Example script | PyFrag Plotter
This notebook acts as an example of how to use the "pyfrag_plotter" package to plot the results of a PyFrag calculation.

### Prequirements:
- Installation of the "pyfrag_plotter" package
> `python -m pip install pyfrag_plotter`

### Requirements for running the script: 
- Directory containing the relevant PyFrag files, being the inputfile (.in) and the outputfile (pyfrag_*.txt)

### Flow of the program:
- Set up paths to the relevant files and define the irc label (instrinic reaction coordinate)
- Initialize the config file
- Make PyFrag objects
- Use the PyFrag objects to initialize the PyFragPlotter object
- Plot the results!

## Importing modules

In [ ]:
# Load the modules
import os
from pyfrag_plotter import initialize_pyfrag_plotter
from pyfrag_plotter.pyfrag_object import create_pyfrag_object_from_dir
from pyfrag_plotter.plot.plotter import Plotter

# Setting up paths and variables

Next, we specify the following paths:
- pyfrag_dir: the directory where the pyfrag output file is located. For example, the .in and .txt file are located in pyfrag_dir/[calculation1]/*.in
- plot_dir: the directory where the plots will be saved
- config_file: path to the config file that we use for initializing the pyfrag_plotter package

For this example, we assume that the pyfrag directory (the directory containing the calcultions), the plot_dir and config file is in the same directory as this notebook

In [ ]:
current_dir_path = os.getcwd()
pyfrag_dir = current_dir_path
plot_dir = os.path.join(current_dir_path, "example_plots")
config_file = os.path.join(current_dir_path, "example_config.ini")

result_dirs = ["ureas_di_O_Cs_all", "ureas_di_O_Cs_pi", "ureas_di_O_Cs_sigma",]
pyfrag_dirs = [os.path.join(pyfrag_dir, directory) for directory in result_dirs]

# Below specifies the irc label, being the key of the irc coordinate as found in the .txt file, and the corresponding label for the x-axis in the plots.
irc_coord = ("bondlength_1", "r - r$_{eq}$ / Å")

## Initializing the program with a config file

With that all set, we can initialize the pyfrag_plotter package with the initialize_pyfrag_plotter function using the config file. In this file, you can specify plot settings, such as the font, font size, figure size, etc, but also which eda keys to plot, and until which energy value (dEint) to plot.

If you have a specific config file you want to use, you can provide it as an argument to the initialize_pyfrag_plotter function. If you don't provide a config file, the default config file will be used that is part of the package.

In [ ]:
# First, initialize the config file
initialize_pyfrag_plotter(user_config_file=config_file)


## Creating the PyFrag object(s) 

The settings are now internally loaded. We can move on to loading the data. We do this with the create_pyfrag_object_from_dir. This function takes the pyfrag_dir as an argument, loads the *in and pyfrag_*txt files, and returns pyfrag object.

Under the hood the data in the .txt file is loaded into a pandas dataframe, and the data in the .in file is loaded into a dictionary. The dataframe is processed by removing outliers, trimming the data, and removing the dispersion key. Try supplying the following arguments in the create_pyfrag_object_from_dir function:
- trim_option: until which point, or between which domain the data should be trimmed to. Options are "max", "min", "x_limits", or "none"
- trim_key: the key to trim on, for example "EnergyTotal", or "bondlength_1" if you also supply the trim_option "x_limits"
- outlier_threshold: the threshold for removing outliers. For example, 50 means that all values that are more than 50 away from the two nearest values are removed. This is handy for removing sudden jumps in the data, which can happen in the case of spin correlation issues.

In [ ]:
objs = [create_pyfrag_object_from_dir(pyfrag_dir) for pyfrag_dir in pyfrag_dirs]

# Plotting

Now let's do some plotting! 

Make an instance of the MultiPlotter by providing:
- name: how do you want to name the plot? The name specifies the name of the folder in which the plots are stored. 
- plot_dir: the path to directory in which the plots are stored.
- pyfrag_objects: the PyFragResultsObjects (list) 
- irc_coord = irc coordinate (tuple of (irc_key (str), label (str)) on which the data should be plotted with the corresponding x-axis label. 

The Multiplotter is able to plot ASM, EDA and ASM extra strain curves. You can even provide the keys that should be plotted, such as "EnergyTotal" or "Int" (they should match the keys in the pyfrag_*.txt resultsfile).

Try below to add additional keys to plot, such as "EnergyTotal" and "Int". Also try changing values in the config file and see what happens!

For all the options, visit the [Documentation](https://siebelede.github.io/pyfrag_plot/index.html) 

In [ ]:
plot_inst = Plotter(name="O_tri_ureas", plot_dir=plot_dir, pyfrag_objects=objs, irc_coord=("bondlength_1",  "r - r$_{eq}$ / Å"))

# with plot_inst as plotter:
    # plotter.plot_asm()  # ["EnergyTotal"]
    # plotter.plot_asm(["EnergyTotal"], plot_legend=False)  # ["EnergyTotal"]
    # plotter.plot_eda(["Int", "Pauli", "Elstat", "OI"])

You can also plot arbitrary keys by using the `plot_arbitrary_key` function

In [ ]:
with plot_inst as plotter:
    plotter.plot_arbitrary_keys(title="Arbitrary_plot", keys=["bondlength_1"], y_lim=[0, 1])